In [17]:
import nltk
import string
import re
import pandas as pd
import glob
import os
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Retrieving data from files

In [18]:
file_names = []
data = []
for filename in os.listdir("data_folder"):
    file_names.append(filename)
    
    try:
            f = open("data_folder/"+filename , encoding='utf8')
            var=f.read().replace('\n'," ")
            data.append(var)
            
    except Exception as e:
            f = open("data_folder/"+filename , encoding='unicode_escape')
            var=f.read().replace('\n'," ")
            data.append(var)
            
    f.close();
            

# Preprocessing

In [19]:
def preprocessing(d):
    no_punc = []
    d_lower=d.lower()
    
    nltk_tokens = nltk.word_tokenize(d_lower)
    
    stop_words_removed = []
    for w in nltk_tokens:
        if w not in stop_words:
            stop_words_removed.append(w)
    
    
    new_words = []
    for x in stop_words_removed:
        if(x.isalnum() and x!=" "):
            new_words.append(x)
        
        
    return new_words
       

In [20]:
preprocessed_data = []
for d in data:
    data_new = preprocessing(d)
    preprocessed_data.append(data_new)    

# Creating Positional indexing

In [21]:
def positional_indexing(dict_position , file_id  , data):
    
    for position , token in enumerate(data):
        
        
        if token in dict_position:
            dict_position[token][0] +=1
            if file_id in dict_position[token][1]:
                dict_position[token][1][file_id].append(position)
                
            else:
                dict_position[token][1][file_id] = [position]
                
        else:
            new_dict = {}
            new_dict[file_id] = [position]
            dict_position[token] = [1 , new_dict]
            
        
    return dict_position
                
        
        
        
    

In [22]:
dict_position = {}
for i in range(len(preprocessed_data)):
     dict_position = positional_indexing(dict_position , i  , preprocessed_data[i])

# Loading positional indexing datastructure

In [23]:
def load_datastructure():
    ds = open("postings2.pkl" , "rb")
    postings = pickle.load(ds)
    
    return postings

In [24]:
def perform_stemming(q):
    stemmer = PorterStemmer()
    stemmed = []
    for word in q:
        stemmed.append(stemmer.stem(word))
    return stemmed  

# Saving positional indexing datastructure

In [25]:
import pickle
posting_datastructure = open("postings2.pkl" , "wb")
pickle.dump(dict_position ,posting_datastructure )
posting_datastructure.close()

In [26]:
def intersection(posting1 , posting2 , diff):
    len1 = len(posting1[1].keys())
    len2 = len(posting2[1].keys())
    #len1 = posting1[0]
    #len2 = posting2[0]
    doc_ids = []
    
    if(len1<=len2):
        ps1 = posting1
        ps2 = posting2
    
    else:
        ps1 = posting2
        ps2 = posting1
        diff = -diff
    
    for file_id in ps1[1].keys():
        if file_id in ps2[1].keys():
            
            for pos1 in ps1[1][file_id]:
                for pos2 in ps2[1][file_id]:
                    if pos2-pos1== diff :
                        if file_id not in doc_ids:
                            doc_ids.append(file_id)
                        break
    
    return doc_ids
                        
               

# Searching Phrase Query

In [27]:
def search_phrase_query(query):
    
    posting_ds = load_datastructure()
    no_phrase=0
    flag = 0
    postings = []
    for word in query :
        if word in posting_ds:
            postings.append(posting_ds[word])
        else:
            flag=1
            break
            
    alldocs = []
    if(flag==1):
        print("1")
        print("Given phrase is not present")
    else:
        for i in range(len(postings)-1):
            j=i+1
            for j in range(len(postings)):
                docs = []
                x = j-i
                docs = intersection(postings[i], postings[j] , x)
                if(len(docs)!=0):
                    alldocs.append(docs)
                else:
                    print("2")
                    print("Given phrase is not present")
                    no_phrase=1
                    break
            if no_phrase == 1:
                break
            
        if no_phrase == 0:
            final_result = set.intersection(*[set(x) for x in alldocs])
            end_result = []
            end_result = list(final_result)
            if(len(end_result) == 0):
                print("3")
                print("Given phrase is not present")
            else:
                print("Number of documents retrieved:")
                print(len(end_result))
                print("Names of documents are:")
                for f in end_result:
                    print(file_names[f])
        
                return end_result
        else:
            end_result="nothing"
            return end_result
        

In [31]:
query = input("Please enter your query")
query = preprocessing(query)
if(len(query)==1):
    posting_ds = load_datastructure()
    word = query[0]
    if word in posting_ds:
        d = posting_ds[word][1].keys()
        d = list(d)
        print("Number of documents retrieved:")
        print(len(d))
        print("Names of documents are:")
        for f in d:
            print(file_names[f])
    else:
        print("phrase not present!")
else:
        search_phrase_query(query)

Please enter your queryWoods World Championship Chili
Number of documents retrieved:
2
Names of documents are:
woods.txt
chili.txt
